
# TA-Lib Candlestick **Scanner + Confirmations** (Swing Trading, Daily)
**What it does**
1. Downloads daily OHLCV for a list of tickers (e.g., NSE: `RELIANCE.NS`, `TCS.NS`, `HDFCBANK.NS`, `LICI.NS`).  
2. Scans **industry-standard** candlestick patterns using **TA-Lib** (e.g., Engulfing, Morning/Evening Star, Hammer, Shooting Star, Harami, Dark Cloud Cover, Piercing).  
3. **Confirms** the raw pattern using objective signals suitable for swing trading: **RSI, MACD**, **volume spike vs 20-day average**, and **trend filter (SMA-50/200)**.  
4. Outputs a clean CSV of **tradable signals** with suggested **entry triggers**, **stops (ATR-based)**, and key metrics for sizing.

> Tip: Use this scanner to build a watchlist. Then trade only those that **confirm on a close** or **break the pattern high/low** with an ATR buffer.



## 0) Setup
Run this cell once to install dependencies if needed. TA-Lib has prebuilt wheels for most platforms.  
If installation fails on Linux, install system `ta-lib` first (e.g., `apt-get install ta-lib`) then `pip install TA-Lib`.


In [5]:

# If you're missing any package, uncomment the pip installs below and run once.
# %pip install yfinance TA-Lib pandas numpy matplotlib --quiet

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import datetime as dt

try:
    import talib as ta
except Exception as e:
    print("[WARN] TA-Lib not available. Install via: pip install TA-Lib")
    raise

try:
    import yfinance as yf
except Exception as e:
    print("[WARN] yfinance not available. Install via: pip install yfinance")
    raise

import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 160)



## 1) Configuration
- **Tickers:** Any Yahoo Finance symbols. For NSE, use the `.NS` suffix (e.g., `RELIANCE.NS`).
- **Patterns:** Choose from the common, higher-quality TA-Lib patterns.
- **Confirmations:** You can require N-of-M confirmations. Defaults are conservative for swing trading.


In [6]:

# ===== User Parameters =====
TICKERS = ['360ONE.NS', '3MINDIA.NS', 'ABB.NS', 'ACC.NS', 'ACMESOLAR.NS', 'AIAENG.NS', 'APLAPOLLO.NS', 'AUBANK.NS', 'AWL.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABBOTINDIA.NS', 'ACE.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'ATGL.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'AEGISLOG.NS', 'AFCONS.NS', 'AFFLE.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'APLLTD.NS', 'ALIVUS.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'ARE&M.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAZEN.NS', 'ASTRAL.NS', 'ATUL.NS', 'AUROPHARMA.NS', 'AIIL.NS', 'DMART.NS', 'AXISBANK.NS', 'BASF.NS', 'BEML.NS', 'BLS.NS', 'BSE.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BAJAJHLDNG.NS', 'BAJAJHFL.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'MAHABANK.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BERGEPAINT.NS', 'BDL.NS', 'BEL.NS', 'BHARATFORG.NS', 'BHEL.NS', 'BPCL.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BSOFT.NS', 'BLUEDART.NS', 'BLUESTARCO.NS', 'BBTC.NS', 'BOSCHLTD.NS', 'FIRSTCRY.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'MAPMYINDIA.NS', 'CCL.NS', 'CESC.NS', 'CGPOWER.NS', 'CRISIL.NS', 'CAMPUS.NS', 'CANFINHOME.NS', 'CANBK.NS', 'CAPLIPOINT.NS', 'CGCL.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CDSL.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOLAHLDNG.NS', 'CHOLAFIN.NS', 'CIPLA.NS', 'CUB.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CAMS.NS', 'CONCORDBIO.NS', 'CONCOR.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CROMPTON.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DCMSHRIRAM.NS', 'DLF.NS', 'DOMS.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'LALPATHLAB.NS', 'DRREDDY.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'EICHERMOT.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'NYKAA.NS', 'FEDERALBNK.NS', 'FACT.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FSL.NS', 'FIVESTAR.NS', 'FORTIS.NS', 'GAIL.NS', 'GVT&D.NS', 'GMRAIRPORT.NS', 'GRSE.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'MEDANTA.NS', 'GODIGIT.NS', 'GPIL.NS', 'GODFRYPHLP.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GESHIP.NS', 'FLUOROCHEM.NS', 'GUJGASLTD.NS', 'GMDCLTD.NS', 'GNFC.NS', 'GPPL.NS', 'GSPL.NS', 'HEG.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HFCL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HEROMOTOCO.NS', 'HSCL.NS', 'HINDALCO.NS', 'HAL.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'POWERINDIA.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDFCFIRSTB.NS', 'IFCI.NS', 'IIFL.NS', 'INOXINDIA.NS', 'IRB.NS', 'IRCON.NS', 'ITC.NS', 'ITI.NS', 'INDGN.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'IEX.NS', 'INDHOTEL.NS', 'IOC.NS', 'IOB.NS', 'IRCTC.NS', 'IRFC.NS', 'IREDA.NS', 'IGL.NS', 'INDUSTOWER.NS', 'INDUSINDBK.NS', 'NAUKRI.NS', 'INFY.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'INDIGO.NS', 'IGIL.NS', 'IKS.NS', 'IPCALAB.NS', 'JBCHEPHARM.NS', 'JKCEMENT.NS', 'JBMA.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JSWENERGY.NS', 'JSWHL.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JPPOWER.NS', 'J&KBANK.NS', 'JINDALSAW.NS', 'JSL.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JWL.NS', 'JUSTDIAL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KPRMILL.NS', 'KEI.NS', 'KNRCON.NS', 'KPITTECH.NS', 'KAJARIACER.NS', 'KPIL.NS', 'KALYANKJIL.NS', 'KANSAINER.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KFINTECH.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KOTAKBANK.NS', 'KIMS.NS', 'LTF.NS', 'LTTS.NS', 'LICHSGFIN.NS', 'LTFOODS.NS', 'LTIM.NS', 'LT.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LUPIN.NS', 'MMTC.NS', 'MRF.NS', 'MGL.NS', 'MAHSEAMLES.NS', 'M&MFIN.NS', 'M&M.NS', 'MANAPPURAM.NS', 'MRPL.NS', 'MANKIND.NS', 'MARICO.NS', 'MARUTI.NS', 'MASTEK.NS', 'MFSL.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'METROPOLIS.NS', 'MINDACORP.NS', 'MSUMI.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MCX.NS', 'MUTHOOTFIN.NS', 'NATCOPHARM.NS', 'NBCC.NS', 'NCC.NS', 'NHPC.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPCGREEN.NS', 'NTPC.NS', 'NH.NS', 'NATIONALUM.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NETWORK18.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NAM-INDIA.NS', 'NIVABUPA.NS', 'NUVAMA.NS', 'OBEROIRLTY.NS', 'ONGC.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'PAYTM.NS', 'OFSS.NS', 'POLICYBZR.NS', 'PCBL.NS', 'PGEL.NS', 'PIIND.NS', 'PNBHOUSING.NS', 'PNCINFRA.NS', 'PTCIL.NS', 'PVRINOX.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFIZER.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PEL.NS', 'PPLPHARMA.NS', 'POLYMED.NS', 'POLYCAB.NS', 'POONAWALLA.NS', 'PFC.NS', 'POWERGRID.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PNB.NS', 'RRKABEL.NS', 'RBLBANK.NS', 'RECLTD.NS', 'RHIM.NS', 'RITES.NS', 'RADICO.NS', 'RVNL.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RKFORGE.NS', 'RCF.NS', 'RTNINDIA.NS', 'RAYMONDLSL.NS', 'RAYMOND.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RPOWER.NS', 'ROUTE.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SRF.NS', 'SAGILITY.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'MOTHERSON.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'RENUKA.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'STARHEALTH.NS', 'SBIN.NS', 'SAIL.NS', 'SWSOLAR.NS', 'SUMICHEM.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANCORP.NS', 'SWIGGY.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TBOTEK.NS', 'TVSMOTOR.NS', 'TANLA.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TCS.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TTML.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'NIACL.NS', 'RAMCOCEM.NS', 'THERMAX.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TARIL.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRIVENI.NS', 'TRITURBINE.NS', 'TIINDIA.NS', 'UCOBANK.NS', 'UNOMINDA.NS', 'UPL.NS', 'UTIAMC.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UBL.NS', 'UNITDSPR.NS', 'USHAMART.NS', 'VGUARD.NS', 'DBREALTY.NS', 'VTL.NS', 'VBL.NS', 'MANYAVAR.NS', 'VEDL.NS', 'VIJAYA.NS', 'VMM.NS', 'IDEA.NS', 'VOLTAS.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WESTLIFE.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZFCVINDIA.NS', 'ZEEL.NS', 'ZENTEC.NS', 'ZENSARTECH.NS', 'ZYDUSLIFE.NS', 'ECLERX.NS']


START = "2024-01-01"
END   = dt.date.today().isoformat()
INTERVAL = "1d"  # daily swing

# Patterns to scan (industry-standard set). You can add/remove.
PATTERNS = [
    "CDLENGULFING",
    "CDLPIERCING",
    "CDLMORNINGSTAR",
    "CDLEVENINGSTAR",
    "CDLDARKCLOUDCOVER",
    "CDLHARAMI",
    "CDLHARAMICROSS",
    "CDLHAMMER",
    "CDLINVERTEDHAMMER",
    "CDLSHOOTINGSTAR",
    "CDLHANGINGMAN",
    "CDLDOJI",
]

# ---- Confirmation rules ----
# Long (bullish) requires at least this many confirmations:
REQ_BULL_CONF = 4
# Short (bearish) requires at least this many confirmations:
REQ_BEAR_CONF = 4

# Components of confirmation (toggle individually)
USE_RSI = True
RSI_LEN = 14
RSI_LONG_MIN = 50       # RSI must be > 50 for longs
RSI_SHORT_MAX = 50      # RSI must be < 50 for shorts

USE_MACD = True
MACD_FAST = 12; MACD_SLOW = 26; MACD_SIGNAL = 9
MACD_CONFIRM_MODE = "line_cross"  # "line_cross" or "hist_above0"

USE_VOLUME_SPIKE = True
VOL_LOOKBACK = 20
VOL_SPIKE_MULT = 1.5    # vol >= 1.5x 20d avg

USE_TREND_FILTER = True
SMA_SHORT = 50
SMA_LONG  = 200
# Longs prefer Close >= SMA200; shorts prefer Close <= SMA200
# We also record relation to SMA50 for context.

# Entry/stop suggestions (not an order! just helpful columns)
ENTRY_BUFFER_PCT = 0.003          # 0.3% beyond pattern high/low
ATR_LEN = 14
STOP_ATR_MULT = 1.2               # stop beyond pattern low/high by 1.2*ATR
CAPITAL = 100000                  # for sizing helper columns
RISK_PCT = 0.01                   # risk 1% per trade
ALLOW_SHORTS = False              # NSE delivery shorts are non-trivial; keep False by default

SAVE_CSV_PATH = "confirmed_candlestick_signals_talib.csv"



## 2) Helper functions
- `fetch_ohlcv`: gets OHLCV and computes indicators.
- `apply_patterns`: runs TA-Lib candlestick detectors and stacks their signals.
- `confirm_signal`: checks N-of-M confirmations (RSI, MACD, Volume, Trend).
- `size_position`: simple risk-based sizing using ATR.


In [7]:

def fetch_ohlcv(ticker: str, start: str, end: str, interval: str="1d") -> pd.DataFrame:
    df = yf.download(ticker, start=start, end=end, interval=interval, progress=False, auto_adjust=False, multi_level_index=False)
    if df.empty:
        return df
    df.index = pd.to_datetime(df.index).tz_localize(None)
    # Indicators
    df["RSI"] = ta.RSI(df["Close"], timeperiod=RSI_LEN)
    macd, macd_signal, macd_hist = ta.MACD(df["Close"], fastperiod=MACD_FAST, slowperiod=MACD_SLOW, signalperiod=MACD_SIGNAL)
    df["MACD"] = macd
    df["MACD_SIGNAL"] = macd_signal
    df["MACD_HIST"] = macd_hist
    df["SMA50"] = ta.SMA(df["Close"], timeperiod=SMA_SHORT)
    df["SMA200"] = ta.SMA(df["Close"], timeperiod=SMA_LONG)
    df["ATR"] = ta.ATR(df["High"], df["Low"], df["Close"], timeperiod=ATR_LEN)
    df["VOL20"] = df["Volume"].rolling(VOL_LOOKBACK).mean()
    return df

def apply_patterns(df: pd.DataFrame, patterns: list) -> pd.DataFrame:
    out = []
    o,h,l,c = df["Open"], df["High"], df["Low"], df["Close"]
    for pname in patterns:
        if not hasattr(ta, pname):
            print(f"[WARN] TA-Lib has no function {pname}, skipping.")
            continue
        func = getattr(ta, pname)
        vals = func(o,h,l,c)
        sig = df.copy()[["Open","High","Low","Close","Volume","RSI","MACD","MACD_SIGNAL","MACD_HIST","SMA50","SMA200","ATR","VOL20"]]
        sig["pattern"] = pname
        sig["raw_value"] = vals
        # TA-Lib returns +100/+200 (bullish) or -100/-200 (bearish), 0 otherwise
        sig = sig[sig["raw_value"] != 0]
        if not sig.empty:
            out.append(sig)
    if not out:
        return pd.DataFrame()
    return pd.concat(out).sort_index()

def confirm_signal(row: pd.Series, direction: str) -> (bool, list):
    met = []

    if USE_RSI:
        if direction == "bullish" and row["RSI"] > RSI_LONG_MIN:
            met.append("RSI>50")
        if direction == "bearish" and row["RSI"] < RSI_SHORT_MAX:
            met.append("RSI<50")

    if USE_MACD and not np.isnan(row["MACD"]) and not np.isnan(row["MACD_SIGNAL"]):
        if MACD_CONFIRM_MODE == "line_cross":
            if direction == "bullish" and row["MACD"] > row["MACD_SIGNAL"]:
                met.append("MACD>Signal")
            if direction == "bearish" and row["MACD"] < row["MACD_SIGNAL"]:
                met.append("MACD<Signal")
        else:  # hist_above0
            if direction == "bullish" and row["MACD_HIST"] > 0:
                met.append("MACD_hist>0")
            if direction == "bearish" and row["MACD_HIST"] < 0:
                met.append("MACD_hist<0")

    if USE_VOLUME_SPIKE and not np.isnan(row["VOL20"]) and row["VOL20"] > 0:
        if row["Volume"] >= VOL_SPIKE_MULT * row["VOL20"]:
            met.append(f"Vol≥{VOL_SPIKE_MULT}x20d")

    if USE_TREND_FILTER:
        if direction == "bullish" and row["Close"] >= row["SMA200"]:
            met.append("Close≥SMA200")
        if direction == "bearish" and row["Close"] <= row["SMA200"]:
            met.append("Close≤SMA200")

    req = REQ_BULL_CONF if direction == "bullish" else REQ_BEAR_CONF
    return (len(met) >= req, met)

def size_position(entry: float, stop: float, capital: float=CAPITAL, risk_pct: float=RISK_PCT) -> int:
    risk_amount = capital * risk_pct
    per_share_risk = max(entry - stop, stop - entry)
    if per_share_risk <= 0:
        return 0
    shares = int(risk_amount // per_share_risk)
    return max(shares, 0)



## 3) Scan tickers
For each ticker:  
1. Compute indicators  
2. Apply pattern detectors  
3. Keep rows where **pattern ≠ 0** and **confirmations ≥ threshold**  
4. Suggest entries/stops using **pattern high/low ± buffer** and **ATR**  
5. Save results to CSV


In [8]:

all_signals = []

for t in TICKERS:
    print(f"Scanning {t} ...")
    df = fetch_ohlcv(t, START, END, INTERVAL)
    if df.empty:
        print(f"[WARN] No data for {t}")
        continue

    pats = apply_patterns(df, PATTERNS)
    if pats.empty:
        print(f"[INFO] No raw patterns for {t}")
        continue

    pats = pats.assign(ticker=t)
    rows = []
    for idx, r in pats.iterrows():
        direction = "bullish" if r["raw_value"] > 0 else "bearish"

        # Skip shorts if not allowed
        if direction == "bearish" and not ALLOW_SHORTS:
            continue

        ok, met = confirm_signal(r, direction)
        if not ok:
            continue

        # Entry/stop suggestions
        if direction == "bullish":
            entry = r["High"] * (1 + ENTRY_BUFFER_PCT)
            stop_raw = min(r["Low"], r["Close"])  # defend under the candle
            stop = stop_raw - STOP_ATR_MULT * (r["ATR"] if not np.isnan(r["ATR"]) else 0)
            trigger_note = "Buy on breakout > pattern High (+buffer)"
        else:
            entry = r["Low"] * (1 - ENTRY_BUFFER_PCT)
            stop_raw = max(r["High"], r["Close"])
            stop = stop_raw + STOP_ATR_MULT * (r["ATR"] if not np.isnan(r["ATR"]) else 0)
            trigger_note = "Sell on breakdown < pattern Low (−buffer)"

        size = size_position(entry, stop, CAPITAL, RISK_PCT)

        rows.append({
            "ticker": t,
            "date": idx.date(),
            "pattern": r["pattern"],
            "direction": direction,
            "close": round(r["Close"], 4),
            "high": round(r["High"], 4),
            "low": round(r["Low"], 4),
            "rsi14": round(r["RSI"], 2) if not np.isnan(r["RSI"]) else np.nan,
            "macd": round(r["MACD"], 4) if not np.isnan(r["MACD"]) else np.nan,
            "macd_signal": round(r["MACD_SIGNAL"], 4) if not np.isnan(r["MACD_SIGNAL"]) else np.nan,
            "macd_hist": round(r["MACD_HIST"], 4) if not np.isnan(r["MACD_HIST"]) else np.nan,
            "vol": int(r["Volume"]),
            "vol20": int(r["VOL20"]) if not np.isnan(r["VOL20"]) else np.nan,
            "sma50": round(r["SMA50"], 4) if not np.isnan(r["SMA50"]) else np.nan,
            "sma200": round(r["SMA200"], 4) if not np.isnan(r["SMA200"]) else np.nan,
            "atr14": round(r["ATR"], 4) if not np.isnan(r["ATR"]) else np.nan,
            "confirmations_met": ",".join(met),
            "num_conf": len(met),
            "entry_suggest": round(entry, 4),
            "stop_suggest": round(stop, 4),
            "risk_per_share": round(abs(entry - stop), 4),
            "size_for_1pct": int(size),
            "note": trigger_note
        })
    if rows:
        all_signals.append(pd.DataFrame(rows))

if all_signals:
    signals_df = pd.concat(all_signals).sort_values(["date","ticker"]).reset_index(drop=True)
else:
    signals_df = pd.DataFrame(columns=[
        "ticker","date","pattern","direction","close","high","low","rsi14","macd","macd_signal","macd_hist",
        "vol","vol20","sma50","sma200","atr14","confirmations_met","num_conf","entry_suggest","stop_suggest",
        "risk_per_share","size_for_1pct","note"
    ])

print(f"Found {len(signals_df)} confirmed signals.")
display(signals_df.tail(20))


Scanning 360ONE.NS ...
Scanning 3MINDIA.NS ...
Scanning ABB.NS ...
Scanning ACC.NS ...
Scanning ACMESOLAR.NS ...
Scanning AIAENG.NS ...
Scanning APLAPOLLO.NS ...
Scanning AUBANK.NS ...
Scanning AWL.NS ...
Scanning AADHARHFC.NS ...
Scanning AARTIIND.NS ...
Scanning AAVAS.NS ...
Scanning ABBOTINDIA.NS ...
Scanning ACE.NS ...
Scanning ADANIENSOL.NS ...
Scanning ADANIENT.NS ...
Scanning ADANIGREEN.NS ...
Scanning ADANIPORTS.NS ...
Scanning ADANIPOWER.NS ...
Scanning ATGL.NS ...
Scanning ABCAPITAL.NS ...
Scanning ABFRL.NS ...
Scanning ABREL.NS ...
Scanning ABSLAMC.NS ...
Scanning AEGISLOG.NS ...
Scanning AFCONS.NS ...
Scanning AFFLE.NS ...
Scanning AJANTPHARM.NS ...
Scanning AKUMS.NS ...
Scanning APLLTD.NS ...
Scanning ALIVUS.NS ...
Scanning ALKEM.NS ...
Scanning ALKYLAMINE.NS ...
Scanning ALOKINDS.NS ...
Scanning ARE&M.NS ...
Scanning AMBER.NS ...
Scanning AMBUJACEM.NS ...
Scanning ANANDRATHI.NS ...
Scanning ANANTRAJ.NS ...
Scanning ANGELONE.NS ...
Scanning APARINDS.NS ...
Scanning APOLLOH

,ticker,date,pattern,direction,close,high,low,rsi14,macd,macd_signal,macd_hist,vol,vol20,sma50,sma200,atr14,confirmations_met,num_conf,entry_suggest,stop_suggest,risk_per_share,size_for_1pct,note
628,SUNDRMFAST.NS,2025-09-04,CDLDOJI,bullish,1016.7000,1029.0000,1013.90,57.19,7.9834,2.2141,5.7693,438121,116862,1002.9450,1005.6802,24.4259,"RSI>50,MACD>Signal,Vol≥1.5x20d,Close≥SMA200",4,1032.0870,984.5889,47.4981,21,Buy on breakout > pattern High (+buffer)
629,SUNDRMFAST.NS,2025-09-04,CDLHARAMICROSS,bullish,1016.7000,1029.0000,1013.90,57.19,7.9834,2.2141,5.7693,438121,116862,1002.9450,1005.6802,24.4259,"RSI>50,MACD>Signal,Vol≥1.5x20d,Close≥SMA200",4,1032.0870,984.5889,47.4981,21,Buy on breakout > pattern High (+buffer)
630,GMDCLTD.NS,2025-09-05,CDLENGULFING,bullish,509.0000,515.0000,451.90,73.40,14.1404,6.8470,7.2933,34448673,5042336,416.1350,343.0666,22.2580,"RSI>50,MACD>Signal,Vol≥1.5x20d,Close≥SMA200",4,516.5450,425.1904,91.3546,10,Buy on breakout > pattern High (+buffer)
631,PVRINOX.NS,2025-09-05,CDLENGULFING,bullish,1146.3000,1166.0000,1109.20,68.24,28.5432,28.5198,0.0234,938356,559185,1037.6260,1082.5612,33.4235,"RSI>50,MACD>Signal,Vol≥1.5x20d,Close≥SMA200",4,1169.4980,1069.0917,100.4063,9,Buy on breakout > pattern High (+buffer)
632,RBLBANK.NS,2025-09-05,CDLENGULFING,bullish,276.0000,279.0000,265.80,64.25,3.5632,1.9656,1.5976,26498078,13097545,258.9162,198.8465,8.4203,"RSI>50,MACD>Signal,Vol≥1.5x20d,Close≥SMA200",4,279.8370,255.6956,24.1414,41,Buy on breakout > pattern High (+buffer)
633,SYRMA.NS,2025-09-05,CDLDOJI,bullish,831.2500,872.0000,822.85,75.18,29.4829,21.2219,8.2611,3044525,1426076,704.9390,562.0343,32.6403,"RSI>50,MACD>Signal,Vol≥1.5x20d,Close≥SMA200",4,874.6160,783.6817,90.9343,10,Buy on breakout > pattern High (+buffer)
634,ACMESOLAR.NS,2025-09-08,CDLDOJI,bullish,302.2500,315.0000,301.00,61.64,5.2762,4.4230,0.8533,3971486,1672675,279.1818,238.9561,11.9803,"RSI>50,MACD>Signal,Vol≥1.5x20d,Close≥SMA200",4,315.9450,286.6237,29.3213,34,Buy on breakout > pattern High (+buffer)
635,ELECON.NS,2025-09-08,CDLENGULFING,bullish,569.3000,573.8000,553.80,51.40,-6.3682,-9.0931,2.7250,354714,233761,585.9300,567.4182,16.1201,"RSI>50,MACD>Signal,Vol≥1.5x20d,Close≥SMA200",4,575.5214,534.4559,41.0655,24,Buy on breakout > pattern High (+buffer)
636,DEVYANI.NS,2025-09-09,CDLENGULFING,bullish,186.3200,189.0000,179.00,73.45,4.1524,2.6194,1.5330,7375538,4890579,169.5770,170.3755,5.1429,"RSI>50,MACD>Signal,Vol≥1.5x20d,Close≥SMA200",4,189.5670,172.8286,16.7384,59,Buy on breakout > pattern High (+buffer)
637,NMDC.NS,2025-09-09,CDLHAMMER,bullish,74.6600,75.0900,73.61,63.52,0.9067,0.4053,0.5015,39250424,20406821,70.7494,68.9894,1.5709,"RSI>50,MACD>Signal,Vol≥1.5x20d,Close≥SMA200",4,75.3153,71.7249,3.5903,278,Buy on breakout > pattern High (+buffer)



## 4) Save results
The CSV contains one row per **confirmed** pattern signal with suggested entry/stop and sizing helper columns.


In [9]:

if not signals_df.empty:
    signals_df.to_csv(SAVE_CSV_PATH, index=False)
    print("Saved ->", SAVE_CSV_PATH)
else:
    print("[INFO] No confirmed signals to save. Try loosening confirmations (e.g., REQ_BULL_CONF=2), adding tickers, or extending the date range.")


Saved -> confirmed_candlestick_signals_talib.csv



## 5) Optional: Quick chart for a specific signal
Run the cell, then call `plot_signal("RELIANCE.NS", "2024-08-01")` using a date that appears in your CSV.


In [10]:

def plot_signal(ticker: str, date_str: str, window: int=80):
    df = fetch_ohlcv(ticker, START, END, INTERVAL)
    if df.empty:
        print("No data.")
        return
    d = pd.to_datetime(date_str)
    if d not in df.index:
        print("Date not in data index. Use a date from the CSV.")
        return
    i = df.index.get_loc(d)
    start_i = max(0, i - window)
    sub = df.iloc[start_i:i+window]

    fig = plt.figure(figsize=(12,5))
    plt.title(f"{ticker} around {date_str}")
    plt.plot(sub.index, sub["Close"], label="Close")
    plt.plot(sub.index, sub["SMA50"], label="SMA50")
    plt.plot(sub.index, sub["SMA200"], label="SMA200")
    plt.legend()
    plt.xlabel("Date"); plt.ylabel("Price")
    plt.show()

# Example (uncomment and adjust a date that appears in your CSV):
# plot_signal("RELIANCE.NS", "2024-08-01")
